# Bug triage with Deep Learning - DWEN

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from __future__ import print_function, division

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers

In [5]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [6]:
MAX_SEQUENCE_LENGTH_T = 100 # 40
MAX_SEQUENCE_LENGTH_D = 500 # 200
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

### Parse bugs preproprecessed

In [7]:
# Domain to use
DOMAIN = 'eclipse'
METHOD = 'baseline_dwen'
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = '{}_feature@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_feature_@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [8]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [9]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [10]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


361006

#### Dicionário de títulos e descrições

In [11]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 3min 22s, sys: 7.89 s, total: 3min 30s
Wall time: 4min 26s


#### Hashing bugs by buckets

In [12]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [ ]:
experiment.prepare_dataset(issues_by_buckets)
# Read and create the test queries duplicates
retrieval.create_queries()

Reading train data


In [ ]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

### Generating the batch test

In [ ]:
"Train ", len(baseline.dup_sets_train)

In [ ]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, valid_sim = baseline.batch_iterator(baseline.train_data, 
                                                                                          baseline.dup_sets_train, 
                                                                                          batch_size_test, 1)

valid_title_sample_a = np.concatenate([valid_input_sample['title'], valid_input_sample['title']], 0)
valid_title_sample_b = np.concatenate([valid_input_pos['title'], valid_input_neg['title']], 0)
valid_desc_sample_a = np.concatenate([valid_input_sample['description'], valid_input_sample['description']], 0)
valid_desc_sample_b = np.concatenate([valid_input_pos['description'], valid_input_neg['description']], 0)

test_gen = ([valid_title_sample_a, valid_title_sample_b, valid_desc_sample_a, valid_desc_sample_b], valid_sim)

# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description'].shape[1]

In [ ]:
valid_input_sample['title'].shape, valid_input_sample['description'].shape, valid_sim.shape

### Validar entrada

In [ ]:
%%time 

baseline.display_batch(baseline.train_data, baseline.dup_sets_train, 5)

## Pre-trained embeddings

Loading pretrained word vectors

### Fasttext

In [ ]:
vocab = baseline.load_vocabulary(os.path.join(DIR, 'vocab_embed_fasttext.pkl'))
#print(np.random.choice(vocab, 10))
# for token in vocab:
#     print(token)

In [ ]:
"Total vocabulary: {}".format(len(vocab))

In [ ]:
import io

def generating_embed(baseline, EMBED_DIR, EMBEDDING_DIM):
    embeddings_index = {}
    embed_path = os.path.join(EMBED_DIR, 'crawl-300d-2M.vec')
    f = open(embed_path, 'rb')
    f = io.open(embed_path, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, f.readline().split())

    vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed_fasttext.pkl'))
    vocab_size = len(vocab) 

    # Initialize uniform the vector considering the Tanh activation
    embedding_matrix = np.random.uniform(-1.0, 1.0, (vocab_size, EMBEDDING_DIM))
    embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

    loop = tqdm(f)
    loop.set_description("Loading FastText")
    for line in loop:
        tokens = line.rstrip().split(' ')
        embed = list(map(float, tokens[1:]))
        word = tokens[0]
        embeddings_index[word] = np.asarray(embed, dtype='float32')
        loop.update(1)
    f.close()
    loop.close()

    print('Total %s word vectors in FastText 42B 300d.' % len(embeddings_index))

    loop = tqdm(total=vocab_size)
    loop.set_description('Loading embedding from dataset pretrained')
    i = 0
    for word, embed in vocab.items():
        if word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
        else:
            embedding_matrix[i] = np.asarray(embed, dtype='float32')
        loop.update(1)
        i+=1
    loop.close()
    baseline.embedding_matrix = embedding_matrix

In [22]:
%%time

generating_embed(baseline, EMBED_DIR=EMBED_DIR, EMBEDDING_DIM=EMBEDDING_DIM) # MAX_NB_WORDS=MAX_NB_WORDS


Total 1999995 word vectors in FastText 42B 300d.



CPU times: user 2min 52s, sys: 4.45 s, total: 2min 56s
Wall time: 2min 58s


## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [23]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### Embedding layer

In [24]:
from keras.constraints import MaxNorm
from keras.initializers import TruncatedNormal, RandomUniform

# Is missing the padding_idx used in pytorch
# https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html
# https://stackoverflow.com/questions/54824768/rnn-model-gru-of-word2vec-to-regression-not-learning
def embedding_layer(embeddings, num_words, embedding_dim, max_sequence_length, trainable, name):
    embedding_layer = Embedding(num_words,
                                  embedding_dim,
                                  name='embedding_layer_{}'.format(name),
                                  weights=[embeddings],
                                  embeddings_constraint=MaxNorm(max_value=1, axis=0),
                                  #input_length=max_sequence_length,
                                  input_length=None,
                                  trainable=trainable)
    return embedding_layer

### DWEN model

In [25]:
from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum, Subtract, Average, GlobalAveragePooling1D
from keras.optimizers import Adam, Nadam
import keras.backend as K

def dwen_model(title_a_feature_model, title_b_feature_model, desc_a_feature_model, desc_b_feature_model, \
                  sequence_length_t, sequence_length_d, name):
  
    bug_t_a = Input(shape = (sequence_length_t, ), name = 'title_a_{}'.format(name))
    bug_t_b = Input(shape = (sequence_length_t, ), name = 'title_b_{}'.format(name))
    bug_d_a = Input(shape = (sequence_length_d, ), name = 'desc_a_{}'.format(name))
    bug_d_b = Input(shape = (sequence_length_d, ), name = 'desc_b_{}'.format(name))
    
    # Embedding feature
    bug_t_a_feat = title_a_feature_model(bug_t_a)
    bug_t_b_feat = title_b_feature_model(bug_t_b)
    bug_d_a_feat = desc_a_feature_model(bug_d_a)
    bug_d_b_feat = desc_b_feature_model(bug_d_b)
    
    bug_t_a_feat = GlobalAveragePooling1D()(bug_t_a_feat)
    bug_t_b_feat = GlobalAveragePooling1D()(bug_t_b_feat)
    bug_d_a_feat = GlobalAveragePooling1D()(bug_d_a_feat)
    bug_d_b_feat = GlobalAveragePooling1D()(bug_d_b_feat)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output_a = concatenate([bug_t_a_feat, bug_d_a_feat], name = 'merge_a_features_{}'.format(name))
    bug_feature_output_b = concatenate([bug_t_b_feat, bug_d_b_feat], name = 'merge_b_features_{}'.format(name))
    
    bug_feature_output = concatenate([bug_feature_output_a, bug_feature_output_b])
    
    number_of_units = K.int_shape(bug_feature_output)[1]
    hidden_layers = 2
    
    # Deep Hidden MLPs
    for _ in range(hidden_layers):
        bug_feature_output = Dense(number_of_units // 2)(bug_feature_output)
    
    bug_feature_model = Model(inputs=[bug_t_a, bug_t_b, bug_d_a, bug_d_b], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [31]:
%%time

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Embeddings
desc_a_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_D, 
                              trainable=False, name='desc_a')
desc_b_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_D, 
                              trainable=False, name='desc_b')
title_a_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_T, 
                              trainable=False, name='title_a')
title_b_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_T, 
                              trainable=False, name='title_b')

# Similarity model
feature_model = dwen_model(title_a_embedding_layer, title_b_embedding_layer, 
                                 desc_a_embedding_layer, desc_b_embedding_layer, 
                                    MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'dwen')

 # Sigmoid
output = Dense(1, activation='sigmoid')(feature_model.output)

similarity_model = Model(inputs=feature_model.input, outputs=[output], name = 'dwen_output')

#optimizer = Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.01)
optimizer = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

# setup the optimization process 
similarity_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()


'''
    Configuration
'''
epochs = 100
best_loss = float('inf')
best_epoch = 0
verbose = 0
loss = 0

'''
    Experiment
'''
for epoch in range(epochs):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, \
            train_sim = baseline.batch_iterator(baseline.train_data, baseline.dup_sets_train, batch_size, 1)
    
    num_batch = train_input_sample['title'].shape[0]
    pos = np.full((1, num_batch), 1)
    neg = np.full((1, num_batch), 0)
    train_sim = np.concatenate([pos, neg], -1)[0]
    title_sample_a = np.concatenate([train_input_sample['title'], train_input_sample['title']], 0)
    title_sample_b = np.concatenate([train_input_pos['title'], train_input_neg['title']], 0)
    desc_sample_a = np.concatenate([train_input_sample['description'], train_input_sample['description']], 0)
    desc_sample_b = np.concatenate([train_input_pos['description'], train_input_neg['description']], 0)
    train_batch = [title_sample_a, title_sample_b, desc_sample_a, desc_sample_b]
    
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, _ = experiment.evaluate_validation_test(retrieval, verbose, feature_model, issues_by_buckets)
        print("Epoch: {} Loss: {:.2f}, acc: {:.2f}, recall@25: {:.2f}".format(epoch+1, h[0],  h[1], recall))
    else:
        print("Epoch: {} Loss: {:.2f}, acc: {:.2f}".format(epoch+1, h[0],  h[1]))
    
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_a_dwen (InputLayer)       (None, 47)           0                                            
__________________________________________________________________________________________________
desc_a_dwen (InputLayer)        (None, 500)          0                                            
__________________________________________________________________________________________________
title_b_dwen (InputLayer)       (None, 47)           0                                            
__________________________________________________________________________________________________
desc_b_dwen (InputLayer)        (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 4 array(s), but instead got the following list of 3 arrays: [array([[ 479,  601,  731, ...,    0,    0,    0],
       [1797,  980,   30, ...,    0,    0,    0],
       [1257, 1752,  111, ...,    0,    0,    0],
       ...,
       [ 227, 3213,  245, ...,    0, ...

In [ ]:
_[:20]

In [ ]:
recall, exported_rank = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets)

"recall@25 last epoch:", recall

### Retrieval evaluation

In [ ]:
print("Total of queries:", len(retrieval.test))

#### Getting the model trained

In [ ]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

In [ ]:
model = experiment.get_model_vectorizer(path=SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)))

In [ ]:
model.summary()

In [ ]:
recall, exported_rank = experiment.evaluate_validation_test(retrieval, 0, model, issues_by_buckets)

In [ ]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

In [ ]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [ ]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report